#### Downlad the dataset

In [ ]:
#Download both images and annotations
!wget -q http://www.robots.ox.ac.uk/~vgg/data/pets/data/images.tar.gz
!wget -q http://www.robots.ox.ac.uk/~vgg/data/pets/data/annotations.tar.gz

In [ ]:
#Check current directory to make sure data is downloaded
!ls -l

In [ ]:
#unzip the tar files downloaded abve
!tar xf images.tar.gz
!tar xf annotations.tar.gz

In [ ]:
#Explore directories
!ls -l

In [ ]:
!ls -l images

In [ ]:
!ls -l images | wc -l

In [ ]:
!ls -l annotations

In [ ]:
!ls -l annotations/xmls

In [ ]:
#Check the xml annotations
!ls -l annotations/xmls | wc -l

In [ ]:
import glob

In [ ]:
len(glob.glob('annotations/xmls/*.xml'))

In [ ]:
!cat annotations/xmls/yorkshire_terrier_18.xml

In [ ]:
#Install tidy to review xml files
!sudo apt-get install tidy --quiet

In [ ]:
#Check one of the xml file to understand annotations
!tidy -xml -i annotations/xmls/wheaten_terrier_170.xml

#### Convert XML to CSV

In [ ]:
#Mount Google drive (change code for local machine). We need to copy generate_dataset.py script to current directory
from google.colab import drive
drive.mount('/gdrive')

In [ ]:
#Copy generate_dataset.py file to current directory
!cp "/content/drive/MyDrive/Deep learning Mtech Residency rev+LI/DL Materials Sagarika/RK content/ACV-I/Notebooks/Localization/generate_dataset.py" .

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!ls -l

In [ ]:
#Move all xml files to images folder, this is needed for python script used next
!mv annotations/xmls/* images/

In [ ]:
#Build csv file for both training and test dataset
!python generate_dataset.py

In [ ]:
!ls -l

#### Visualize Data

In [ ]:
from matplotlib import pyplot as plt
import cv2
import numpy as np
import pandas as pd

In [ ]:
#Read csv file as pandas dataframe, csv file has no header
train_df = pd.read_csv('train.csv', header=None,
                       names=['File', 'Height','Width','xmin',
                              'ymin','xmax', 'ymax','Class','Label'])

In [ ]:
print(train_df.shape)
train_df.sample(n=5)

In [ ]:
#Create a dictionary to hold label and corresponding class name
num_classes = train_df['Label'].unique()
label_class_dict = dict(zip(train_df['Label'], train_df['Class']))
label_class_dict

In [ ]:
num_classes

In [ ]:
len(num_classes)

In [ ]:
label_class_dict

In [ ]:
train_df.shape

Show images with bounding box

In [ ]:
#Pickup a random image number
img_num = np.random.randint(0, train_df.shape[0])
img_num = 832
#Read the image and draw a rectangle as per bounding box information
img = cv2.imread(train_df.loc[img_num,'File'])
img = cv2.resize(img,(224, 224))
w = train_df.loc[img_num, 'Width']
h = train_df.loc[img_num, 'Height']
x_ratio = 224/w
y_ratio = 224/h

cv2.rectangle(img,
             (int(train_df.loc[img_num, 'xmin']*x_ratio),int(train_df.loc[img_num, 'ymin']*y_ratio)),
             (int(train_df.loc[img_num, 'xmax']*x_ratio),int(train_df.loc[img_num, 'ymax']*y_ratio)),
             (0,255,0),
             2)

#Convert BGR format (used by opencv to RGB format used by matplotlib)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

#Draw image using matplotlib
print(train_df.loc[img_num, 'Class'])
plt.figure(figsize=(10,10))
plt.imshow(img)
plt.show()

In [ ]:
img_num

In [ ]:
train_df.loc[img_num, :]

In [ ]:
#Read the validation csv file
test_df = pd.read_csv('validation.csv', header=None,
                       names=['File', 'Height','Width','xmin',
                              'ymin','xmax', 'ymax','Class','Label'])

In [ ]:
train_df.shape, test_df.shape

#### Build a Batch Generator

In [ ]:
import tensorflow as tf

In [ ]:
img_size = 224

In [ ]:
def batch_generator(df, batch_size=32):

    while True:

        #Create indexes
        image_nums = np.random.randint(0,df.shape[0], size=batch_size)

        #Create empty arrays
        #1. To hold image input
        batch_images = np.zeros(shape=(batch_size, img_size, img_size, 3))

        #Classification Labels
        batch_labels = np.zeros(shape=(batch_size, len(num_classes)))

        #Regression labels - 4 numbers per example image
        batch_bboxes = np.zeros(shape=(batch_size, 4))


        for i in range(batch_size):

            #Read image and resize
            img = tf.keras.preprocessing.image.load_img(df.loc[image_nums[i], 'File'],
                                                        target_size=(img_size, img_size))

            #Convert to numpy array
            img_array = tf.keras.preprocessing.image.img_to_array(img)

            #Update batch
            batch_images[i] = img_array

            #Read image classification label & convert to one hot vector
            cl_label = df.loc[image_nums[i], 'Label']
            cl_label = tf.keras.utils.to_categorical(cl_label, num_classes=len(num_classes))
            batch_labels[i] = cl_label

            #Read and resize bounding box co-ordinates
            img_width = df.loc[image_nums[i], 'Width']
            img_height = df.loc[image_nums[i], 'Height']

            xmin = df.loc[image_nums[i], 'xmin'] * img_size/img_width
            xmax = df.loc[image_nums[i], 'xmax'] * img_size/img_width

            ymin = df.loc[image_nums[i], 'ymin'] * img_size/img_height
            ymax = df.loc[image_nums[i], 'ymax'] * img_size/img_height

            #We will ask model to predict xmin, ymin, width and height of bounding box
            batch_bboxes[i] = [xmin, ymin, xmax-xmin, ymax-ymin]

        #Normalize batch images as per Pre-trained model to be used
        for i in range(batch_size):
            batch_images[i] = tf.keras.applications.resnet50.preprocess_input(batch_images[i])

        #Make bounding boxes (x, y, w, h) as numbers between 0 and 1 - this seems to work better
        batch_bboxes = batch_bboxes /img_size

        #Return batch - use yield function to make it a python generator
        yield batch_images, [batch_labels, batch_bboxes]

In [ ]:
gen = batch_generator(train_df, batch_size=20)

In [ ]:
gen

In [ ]:
X, y = next(gen)

In [ ]:
print(X.shape)

In [ ]:
X[0]

In [ ]:
len(y)

In [ ]:
y[0].shape

In [ ]:
#Classification
y[0][0]

In [ ]:
y[1].shape

In [ ]:
#Regression
y[1][0]

#### Build the Model

Load Pre-Trained Model

In [ ]:
tf.keras.backend.clear_session()
model = tf.keras.applications.ResNet50(include_top=False, #Do not include FC layer at the end
                                       input_shape=(img_size, img_size, 3),
                                       weights='imagenet')

In [ ]:
model.summary()

Freeze all layers of Pre-trained model

In [ ]:
len(model.layers)

In [ ]:
for layer in model.layers:
    layer.trainable = False

In [ ]:
model.summary()

In [ ]:
model.output

Add layers

In [ ]:
#get Output layer of Pre-trained model
x1 = model.output

In [ ]:
x1

In [ ]:
#Flatten the output to feed to Dense layer
#x2 = tf.keras.layers.GlobalAveragePooling2D()(x1)
x2 = tf.keras.layers.Conv2D(32, (1,1), activation='relu')(x1)

In [ ]:
x2

In [ ]:
#Add Dropout
#x3 = tf.keras.layers.Dropout(0.5)(x2)
x3 = tf.keras.layers.BatchNormalization()(x2)

In [ ]:
x3

In [ ]:
#Flatten
x4 = tf.keras.layers.Flatten()(x3)

In [ ]:
x4

Build layer for Classification Label output

In [ ]:
#Classification
label_output = tf.keras.layers.Dense(37,
                                     activation='softmax',
                                     name='class_op')(x4)

In [ ]:
label_output

Build layer for bounding box output

In [ ]:
#Regression
bbox_output = tf.keras.layers.Dense(4,
                                    activation='sigmoid',
                                    name='reg_op')(x4)

In [ ]:
bbox_output

Finalize the model

In [ ]:
#Non Sequential model as it has two different outputs
final_model = tf.keras.models.Model(inputs=model.input, #Pre-trained model input as input layer
                                    outputs=[label_output,bbox_output]) #Output layer added

In [ ]:
final_model.summary()

Define function to calculate IoU

In [ ]:
def calculate_iou(y_true, y_pred):


    """
    Input:
    Keras provides the input as numpy arrays with shape (batch_size, num_columns).

    Arguments:
    y_true -- first box, numpy array with format [x, y, width, height, conf_score]
    y_pred -- second box, numpy array with format [x, y, width, height, conf_score]
    x any y are the coordinates of the top left corner of each box.

    Output: IoU of type float32. (This is a ratio. Max is 1. Min is 0.)

    """


    results = []

    for i in range(0,y_true.shape[0]):

        # set the types so we are sure what type we are using
        y_true = np.array(y_true, dtype=np.float32)
        y_pred = np.array(y_pred, dtype=np.float32)

        #print(y_true.shape)
        #print(y_pred.shape)
        # boxTrue
        x_boxTrue_tleft = y_true[i,0]  # numpy index selection
        y_boxTrue_tleft = y_true[i,1]
        boxTrue_width = y_true[i,2]
        boxTrue_height = y_true[i,3]
        area_boxTrue = (boxTrue_width * boxTrue_height)

        # boxPred
        x_boxPred_tleft = y_pred[i,0]
        y_boxPred_tleft = y_pred[i,1]
        boxPred_width = y_pred[i,2]
        boxPred_height = y_pred[i,3]
        area_boxPred = (boxPred_width * boxPred_height)

        # calculate the bottom right coordinates for boxTrue and boxPred

        # boxTrue
        x_boxTrue_br = x_boxTrue_tleft + boxTrue_width
        y_boxTrue_br = y_boxTrue_tleft + boxTrue_height # Version 2 revision

        # boxPred
        x_boxPred_br = x_boxPred_tleft + boxPred_width
        y_boxPred_br = y_boxPred_tleft + boxPred_height # Version 2 revision


        # calculate the top left and bottom right coordinates for the intersection box, boxInt

        # boxInt - top left coords
        x_boxInt_tleft = np.max([x_boxTrue_tleft,x_boxPred_tleft])
        y_boxInt_tleft = np.max([y_boxTrue_tleft,y_boxPred_tleft]) # Version 2 revision

        # boxInt - bottom right coords
        x_boxInt_br = np.min([x_boxTrue_br,x_boxPred_br])
        y_boxInt_br = np.min([y_boxTrue_br,y_boxPred_br])

        # Calculate the area of boxInt, i.e. the area of the intersection
        # between boxTrue and boxPred.
        # The np.max() function forces the intersection area to 0 if the boxes don't overlap.


        # Version 2 revision
        area_of_intersection = \
        np.max([0,(x_boxInt_br - x_boxInt_tleft)]) * np.max([0,(y_boxInt_br - y_boxInt_tleft)])

        iou = area_of_intersection / ((area_boxTrue + area_boxPred) - area_of_intersection)


        # This must match the type used in py_func
        iou = np.array(iou, dtype=np.float32)

        # append the result to a list at the end of each loop
        results.append(iou)

    # return the mean IoU score for the batch
    return np.mean(results)



def IoU(y_true, y_pred):

    # Note: the type float32 is very important. It must be the same type as the output from
    # the python function above or you too may spend many late night hours
    # trying to debug and almost give up.

    iou = tf.py_function(calculate_iou, [y_true, y_pred], tf.float32)

    return iou

In [ ]:
final_model.compile(optimizer='adam',
                    loss={'reg_op':'mse', 'class_op':'categorical_crossentropy'},
                    loss_weights={'reg_op':1, 'class_op':1},
                    metrics={'reg_op':[IoU], 'class_op':['accuracy']})

Train the model

In [ ]:
#Create train and test generator
batchsize = 64
train_generator = batch_generator(train_df, batch_size=batchsize) #batchsize can be changed
test_generator = batch_generator(test_df, batch_size=batchsize)

In [ ]:
final_model.fit(train_generator,
                epochs=5,
                steps_per_epoch= train_df.shape[0]//batchsize,
                validation_data=test_generator,
                validation_steps = test_df.shape[0]//batchsize)

In [ ]:
final_model.compile(optimizer='adam',
                    loss={'reg_op':'mse', 'class_op':'categorical_crossentropy'},
                    loss_weights={'reg_op':10, 'class_op':1},
                    metrics={'reg_op':[IoU], 'class_op':['accuracy']})

In [ ]:
final_model.fit(train_generator,
                epochs=10,
                initial_epoch=5,
                steps_per_epoch= train_df.shape[0]//batchsize,
                validation_data=test_generator,
                validation_steps = test_df.shape[0]//batchsize)

In [ ]:
final_model.fit(train_generator,
                epochs=15,
                initial_epoch=10,
                steps_per_epoch= train_df.shape[0]//batchsize,
                validation_data=test_generator,
                validation_steps = test_df.shape[0]//batchsize)

In [ ]:
final_model.save('pet_dataset_localization.h5')

In [ ]:
final_model.input

#### Model Prediction

In [ ]:
def predict_and_draw(image_num, df):

    #Load image
    img = tf.keras.preprocessing.image.load_img(df.loc[image_num, 'File'])
    w, h = img.size

    #Prepare input for model
    #1. Resize image
    img_resized = img.resize((img_size, img_size))
    #2. Conver to array and make it a batch of 1
    input_array = tf.keras.preprocessing.image.img_to_array(img_resized)
    input_array = np.expand_dims(input_array, axis=0)

    #3. Normalize image data
    input_array = tf.keras.applications.resnet50.preprocess_input(input_array)

    #Prediction
    pred = final_model.predict(input_array)
    #Get classification and regression predictions
    label_pred, bbox_pred = pred[0][0], pred[1][0]

    #print('Class label prediction', label_pred)
    #print('Boundary box prediction', bbox_pred)

    #Get Label with highest probability
    pred_class = label_class_dict[np.argmax(label_pred)]

    #Read actual label and bounding box
    act_class = df.loc[image_num, 'Class']
    xmin, ymin, xmax, ymax = df.loc[image_num, ['xmin', 'ymin', 'xmax', 'ymax']]

    print('Actual Label :', act_class, '\nPredicted Label: ', pred_class)

    #Draw bounding boxes - Actual (Red) and Predicted(Green)
    img = cv2.imread(df.loc[image_num, 'File'])

    #Draw actual bounding box - RED
    img = cv2.rectangle(img, (xmin, ymin),
                        (xmax, ymax), (0,0,255), 3)

    #Draw predicted bounding box
    img = cv2.rectangle(img, (int(bbox_pred[0]*w), int(bbox_pred[1]*h)),
                        (int((bbox_pred[0]+bbox_pred[2])*w),
                         int((bbox_pred[1]+bbox_pred[3])*h)), (0,255,0), 3)

    #Display the picture
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    plt.figure(figsize=(10,10))
    plt.imshow(img)
    plt.show()

In [ ]:
#Predict on Test Dataset
image_num = np.random.randint(0, test_df.shape[0])
predict_and_draw(image_num, test_df)

In [ ]:
img = tf.keras.preprocessing.image.load_img(test_df.loc[0, 'File'], target_size=(224,224))
img_array = tf.keras.preprocessing.image.img_to_array(img)

In [ ]:
img_array.shape

In [ ]:
final_model.input

In [ ]:
np.expand_dims(img_array, axis=0).shape

In [ ]:
final_model.predict(np.expand_dims(img_array, axis=0))